# **Distribución % de créditos por segmento (trimestre)**

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "financiamientos" in nombre and "vallarta" in nombre

    #if nombre.startswith("df_") and "tulancingo" in nombre    
    #and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_financiamientos_2019_vallarta',
 'df_financiamientos_2020_vallarta',
 'df_financiamientos_2021_vallarta',
 'df_financiamientos_2022_vallarta',
 'df_financiamientos_2023_vallarta',
 'df_financiamientos_2024_vallarta']

In [2]:
df_financiamientos_2022_pachuca= df_financiamientos_2022_vallarta[['id','mes','monto']]
df_financiamientos_2023_pachuca= df_financiamientos_2023_vallarta[['id','mes','monto']]
df_financiamientos_2024_pachuca= df_financiamientos_2024_vallarta[['id','mes','monto']]

## 2022

In [3]:
# Definir una función para asignar el trimestre según el número de mes
def asignar_trimestre(mes):
    if mes in range(1, 4):
        return 1
    elif mes in range(4, 7):
        return 2
    elif mes in range(7, 10):
        return 3
    else:
        return 4

# Aplicar la función para asignar el trimestre a cada fila
df_financiamientos_2022_pachuca['trimestre'] = df_financiamientos_2022_pachuca['mes'].apply(asignar_trimestre)
df_financiamientos_2022_pachuca

C:\Users\yoe11\AppData\Local\Temp\ipykernel_12776\3153927988.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_financiamientos_2022_pachuca['trimestre'] = df_financiamientos_2022_pachuca['mes'].apply(asignar_trimestre)


,id,mes,monto,trimestre
0,1,1,0.00,1
1,2,1,1271912.14,1
2,3,1,310344.57,1
3,4,1,308885.47,1
4,5,1,447973.48,1
...,...,...,...,...
1269,1270,12,632345.37,4
1270,1271,12,750000.00,4
1271,1272,12,1409680.93,4
1272,1273,12,1290000.00,4


In [4]:
# Definir los rangos de monto de cada segmento
segmentos = {
    "E": (0,  1000000),
    "D": (1000001,  1750000),
    "C": (1750001,  2500000),
    "B": (2500001,  3250000),  
    "A": (3250001,  4000000),
    "S": (4000001, 12000000),
    "L": (12000001, 22000000),
    "ELITE": (22000001, float('inf'))
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
df_financiamientos_2022_pachuca['segmento'] = df_financiamientos_2022_pachuca['monto'].apply(asignar_segmento)
df_financiamientos_2022_pachuca.head()

C:\Users\yoe11\AppData\Local\Temp\ipykernel_12776\2616155040.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_financiamientos_2022_pachuca['segmento'] = df_financiamientos_2022_pachuca['monto'].apply(asignar_segmento)


,id,mes,monto,trimestre,segmento
0,1,1,0.00,1,E
1,2,1,1271912.14,1,D
2,3,1,310344.57,1,E
3,4,1,308885.47,1,E
4,5,1,447973.48,1,E


In [5]:
trimestres_credit_2022_pachuca = df_financiamientos_2022_pachuca[['trimestre','segmento']]
trimestres_credit_2022_pachuca

,trimestre,segmento
0,1,E
1,1,D
2,1,E
3,1,E
4,1,E
...,...,...
1269,4,E
1270,4,E
1271,4,D
1272,4,D


In [6]:
# Calcular el total de registros
total_registros = len(trimestres_credit_2022_pachuca)
print("Total de registros: ", total_registros)
# Calcular la tabla de frecuencias relativas
tabla_frecuencias = pd.pivot_table(trimestres_credit_2022_pachuca, 
                                   index='segmento', 
                                   columns='trimestre', 
                                   aggfunc='size', 
                                   fill_value=0)

# Convertir los valores a porcentajes
tabla_porcentajes = tabla_frecuencias / total_registros * 100
print(tabla_porcentajes)

Total de registros:  1274
trimestre          1          2          3          4
segmento                                             
A           0.235479   0.392465   0.627943   0.627943
B           0.941915   0.706436   0.863422   1.177394
C           2.040816   1.805338   2.119309   1.648352
D           4.081633   3.296703   3.689168   4.631083
E          17.896389  13.029827  14.913658  17.739403
ELITE       0.156986   0.078493   0.156986   0.078493
L           0.156986   0.156986   0.078493   0.313972
S           1.020408   1.648352   1.726845   1.962323


In [7]:
# Calcular el total de registros
total_registros = len(trimestres_credit_2022_pachuca)
print("Total de registros: ", total_registros)

# Calcular la tabla de frecuencias relativas
tabla_frecuencias = pd.pivot_table(trimestres_credit_2022_pachuca, 
                                   index='segmento', 
                                   columns='trimestre', 
                                   aggfunc='size', 
                                   fill_value=0)

# Convertir los valores a porcentajes
tabla_porcentajes = tabla_frecuencias / total_registros * 100
print(tabla_porcentajes)

# Definir colores
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']
orden_segmentos=["E", "D", "C", "B","A", "S", "L",'ELITE' ]

# Crear la figura de la gráfica
fig = go.Figure()
trimestres = tabla_porcentajes.columns

# Reordenar la tabla de porcentajes según el orden de segmentos
tabla_porcentajes = tabla_porcentajes.reindex(orden_segmentos)

# Agregar trazos para cada trimestre
for i, trimestre in enumerate(trimestres):
    fig.add_trace(go.Bar(
        name=f'{trimestre}', 
        x=tabla_porcentajes.index, 
        y=tabla_porcentajes[trimestre], 
        marker_color=colores[i],
        text=tabla_porcentajes[trimestre],  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    ))

# Actualizar diseño de la gráfica
fig.update_layout(
    yaxis=dict(
        title='%',  # Título del eje y
        gridcolor='#dddcda',  # Color de las líneas que dividen los rangos del eje Y         
    ),  
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend_title="Trimestre",
    barmode='group',
    legend=dict(
        orientation='h',  # Orientación horizontal de la leyenda
        yanchor='bottom',  # Anclar al borde inferior
        y=1.02,  # Colocar la leyenda justo debajo del gráfico
        xanchor='right',
        x=1
    ),
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Configuración personalizada para la gráfica
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }

    # Guardar la gráfica como archivo HTML
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

guardar_grafico_como_html(fig, 'g_bar_dist_creditos_segmento_percent_2022', carpeta='assets/graficas')

# Mostrar la gráfica
fig.show()


Total de registros:  1274
trimestre          1          2          3          4
segmento                                             
A           0.235479   0.392465   0.627943   0.627943
B           0.941915   0.706436   0.863422   1.177394
C           2.040816   1.805338   2.119309   1.648352
D           4.081633   3.296703   3.689168   4.631083
E          17.896389  13.029827  14.913658  17.739403
ELITE       0.156986   0.078493   0.156986   0.078493
L           0.156986   0.156986   0.078493   0.313972
S           1.020408   1.648352   1.726845   1.962323


## 2023

In [8]:
# Definir una función para asignar el trimestre según el número de mes
def asignar_trimestre(mes):
    if mes in range(1, 4):
        return 1
    elif mes in range(4, 7):
        return 2
    elif mes in range(7, 10):
        return 3
    else:
        return 4

# Aplicar la función para asignar el trimestre a cada fila
df_financiamientos_2023_pachuca['trimestre'] = df_financiamientos_2023_pachuca['mes'].apply(asignar_trimestre)
df_financiamientos_2023_pachuca

C:\Users\yoe11\AppData\Local\Temp\ipykernel_12776\1086660624.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,monto,trimestre
0,1,1,2020482.96,1
1,2,1,697386.25,1
2,3,1,1753565.84,1
3,4,1,12918156.00,1
4,5,1,437683.56,1
...,...,...,...,...
1379,1380,12,2945373.00,4
1380,1381,12,600724.91,4
1381,1382,12,863000.00,4
1382,1383,12,0.00,4


In [9]:
# Definir los rangos de monto de cada segmento
segmentos = {
    "E": (0,  1000000),
    "D": (1000001,  1750000),
    "C": (1750001,  2500000),
    "B": (2500001,  3250000),  
    "A": (3250001,  4000000),
    "S": (4000001, 12000000),
    "L": (12000001, 22000000),
    "ELITE": (22000001, float('inf'))
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
df_financiamientos_2023_pachuca['segmento'] = df_financiamientos_2023_pachuca['monto'].apply(asignar_segmento)
df_financiamientos_2023_pachuca.head()

C:\Users\yoe11\AppData\Local\Temp\ipykernel_12776\1725725462.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,monto,trimestre,segmento
0,1,1,2020482.96,1,C
1,2,1,697386.25,1,E
2,3,1,1753565.84,1,C
3,4,1,12918156.00,1,L
4,5,1,437683.56,1,E


In [10]:
trimestres_credit_2023_pachuca = df_financiamientos_2023_pachuca[['trimestre','segmento']]
trimestres_credit_2023_pachuca

,trimestre,segmento
0,1,C
1,1,E
2,1,C
3,1,L
4,1,E
...,...,...
1379,4,B
1380,4,E
1381,4,E
1382,4,E


In [11]:
# Calcular el total de registros
total_registros = len(trimestres_credit_2023_pachuca)
print("Total de registros: ", total_registros)
# Calcular la tabla de frecuencias relativas
tabla_frecuencias = pd.pivot_table(trimestres_credit_2023_pachuca, 
                                   index='segmento', 
                                   columns='trimestre', 
                                   aggfunc='size', 
                                   fill_value=0)

# Convertir los valores a porcentajes
tabla_porcentajes = tabla_frecuencias / total_registros * 100
print(tabla_porcentajes)

Total de registros:  1384
trimestre          1          2          3          4
segmento                                             
A           0.722543   0.794798   0.505780   0.361272
B           1.011561   1.300578   1.445087   0.650289
C           2.095376   2.023121   1.445087   1.950867
D           4.841040   4.479769   4.335260   4.768786
E          15.895954  17.196532  13.005780  13.150289
ELITE       0.072254   0.144509   0.072254   0.000000
L           0.072254   0.289017   0.361272   0.361272
S           1.445087   1.372832   1.806358   2.023121


In [12]:


# Definir colores
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']
orden_segmentos=["E", "D", "C", "B","A", "S", "L",'ELITE' ]

# Crear la figura de la gráfica
fig = go.Figure()
trimestres = tabla_porcentajes.columns
# Reordenar la tabla de porcentajes según el orden de segmentos
tabla_porcentajes = tabla_porcentajes.reindex(orden_segmentos)
# Agregar trazos para cada trimestre
for i, trimestre in enumerate(trimestres):
    fig.add_trace(go.Bar(
        name=f'{trimestre}', 
        x=tabla_porcentajes.index, 
        y=tabla_porcentajes[trimestre], 
        marker_color=colores[i],
        text=tabla_porcentajes[trimestre],  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    ))

# Actualizar diseño de la gráfica
fig.update_layout(
    yaxis=dict(
        title='%',  # Título del eje y
        gridcolor='#dddcda',  # Color de las líneas que dividen los rangos del eje Y         
    ),  
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend_title="Trimestre",
    barmode='group',
    legend=dict(
        orientation='h',  # Orientación horizontal de la leyenda
        yanchor='bottom',  # Anclar al borde inferior
        y=1.02,  # Colocar la leyenda justo debajo del gráfico
        xanchor='right',
        x=1
    ),
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
# Configuración personalizada para la gráfica
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }

    # Guardar la gráfica como archivo HTML
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

guardar_grafico_como_html(fig, 'g_bar_dist_creditos_segmento_percent_2023', carpeta='assets/graficas')

# Mostrar la gráfica
fig.show()


## 2024

In [13]:
# Definir una función para asignar el trimestre según el número de mes
def asignar_trimestre(mes):
    if mes in range(1, 4):
        return 1
    elif mes in range(4, 7):
        return 2
    elif mes in range(7, 10):
        return 3
    else:
        return 4

# Aplicar la función para asignar el trimestre a cada fila
df_financiamientos_2024_pachuca['trimestre'] = df_financiamientos_2024_pachuca['mes'].apply(asignar_trimestre)
df_financiamientos_2024_pachuca

C:\Users\yoe11\AppData\Local\Temp\ipykernel_12776\1484986024.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,monto,trimestre
0,1,1,716650.97,1
1,2,1,308304.47,1
2,3,1,7061148.90,1
3,4,1,0.00,1
4,5,1,219888.56,1
...,...,...,...,...
1175,1176,11,2777502.00,4
1176,1177,11,831747.49,4
1177,1178,11,802413.95,4
1178,1179,11,1588000.00,4


In [14]:
# Definir los rangos de monto de cada segmento
segmentos = {
    "E": (0,  1000000),
    "D": (1000001,  1750000),
    "C": (1750001,  2500000),
    "B": (2500001,  3250000),  
    "A": (3250001,  4000000),
    "S": (4000001, 12000000),
    "L": (12000001, 22000000),
    "ELITE": (22000001, float('inf'))
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
df_financiamientos_2024_pachuca['segmento'] = df_financiamientos_2024_pachuca['monto'].apply(asignar_segmento)
df_financiamientos_2024_pachuca.head()

C:\Users\yoe11\AppData\Local\Temp\ipykernel_12776\935610015.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,monto,trimestre,segmento
0,1,1,716650.97,1,E
1,2,1,308304.47,1,E
2,3,1,7061148.90,1,S
3,4,1,0.00,1,E
4,5,1,219888.56,1,E


In [15]:
trimestres_credit_2023_pachuca = df_financiamientos_2024_pachuca[['trimestre','segmento']]
trimestres_credit_2023_pachuca

,trimestre,segmento
0,1,E
1,1,E
2,1,S
3,1,E
4,1,E
...,...,...
1175,4,B
1176,4,E
1177,4,E
1178,4,D


In [16]:
trimestres_credit_2023_pachuca = trimestres_credit_2023_pachuca[trimestres_credit_2023_pachuca['trimestre']!=4]

In [17]:
# Calcular el total de registros
total_registros = len(trimestres_credit_2023_pachuca)
print("Total de registros: ", total_registros)
# Calcular la tabla de frecuencias relativas
tabla_frecuencias = pd.pivot_table(trimestres_credit_2023_pachuca, 
                                   index='segmento', 
                                   columns='trimestre', 
                                   aggfunc='size', 
                                   fill_value=0)

# Convertir los valores a porcentajes
tabla_porcentajes = tabla_frecuencias / total_registros * 100
print(tabla_porcentajes)

Total de registros:  964
trimestre          1          2          3
segmento                                  
A           0.518672   0.933610   0.726141
B           2.074689   1.763485   2.074689
C           3.630705   2.800830   2.800830
D           6.120332   6.639004   6.016598
E          17.427386  19.294606  17.427386
ELITE       0.000000   0.103734   0.207469
L           0.414938   0.414938   0.207469
S           2.800830   3.319502   2.282158


In [18]:


# Definir colores
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']
orden_segmentos=["E", "D", "C", "B","A", "S", "L", 'ELITE']

# Crear la figura de la gráfica
fig = go.Figure()
trimestres = tabla_porcentajes.columns
# Reordenar la tabla de porcentajes según el orden de segmentos
tabla_porcentajes = tabla_porcentajes.reindex(orden_segmentos)

# Agregar trazos para cada trimestre
for i, trimestre in enumerate(trimestres):
    fig.add_trace(go.Bar(
        name=f'{trimestre}', 
        x=tabla_porcentajes.index, 
        y=tabla_porcentajes[trimestre], 
        marker_color=colores[i],
        text=tabla_porcentajes[trimestre],  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    ))

# Actualizar diseño de la gráfica
fig.update_layout(
    yaxis=dict(
        title='%',  # Título del eje y
        gridcolor='#dddcda',  # Color de las líneas que dividen los rangos del eje Y         
    ),  
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend_title="Trimestre",
    barmode='group',
    legend=dict(
        orientation='h',  # Orientación horizontal de la leyenda
        yanchor='bottom',  # Anclar al borde inferior
        y=1.02,  # Colocar la leyenda justo debajo del gráfico
        xanchor='right',
        x=1
    ),
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
# Configuración personalizada para la gráfica
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }

    # Guardar la gráfica como archivo HTML
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

guardar_grafico_como_html(fig, 'g_bar_dist_creditos_segmento_percent_2024', carpeta='assets/graficas')

# Mostrar la gráfica
fig.show()


# DIRECTOS

### 2022

In [14]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica
# Datos originales
data = {
    'segmento': ['E', 'D', 'C', 'B', 'A', 'S', 'L'],
    '1': [17.72, 4.47, 2.00, 0.96, 0.32, 1.04, 0.16],
    '2': [13.65, 3.03, 2.00, 0.80, 0.40, 1.60, 0.16],
    '3': [15.80, 3.83, 2.00, 1.20, 0.48, 1.44, 0.08]
}

# Crear DataFrame
df = pd.DataFrame(data)

# Reorganizar el DataFrame
tabla_porcentajes = df.set_index("segmento")

# Imprimir el DataFrame
print(tabla_porcentajes)


              1      2      3
segmento                     
E         17.72  13.65  15.80
D          4.47   3.03   3.83
C          2.00   2.00   2.00
B          0.96   0.80   1.20
A          0.32   0.40   0.48
S          1.04   1.60   1.44
L          0.16   0.16   0.08


In [15]:


# Definir colores
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']
orden_segmentos=["E", "D", "C", "B","A", "S", "L", 'ELITE']

# Crear la figura de la gráfica
fig = go.Figure()
trimestres = tabla_porcentajes.columns
# Reordenar la tabla de porcentajes según el orden de segmentos
tabla_porcentajes = tabla_porcentajes.reindex(orden_segmentos)

# Agregar trazos para cada trimestre
for i, trimestre in enumerate(trimestres):
    fig.add_trace(go.Bar(
        name=f'{trimestre}', 
        x=tabla_porcentajes.index, 
        y=tabla_porcentajes[trimestre], 
        marker_color=colores[i],
        text=tabla_porcentajes[trimestre],  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    ))

# Actualizar diseño de la gráfica
fig.update_layout(
    yaxis=dict(
        title='%',  # Título del eje y
        gridcolor='#dddcda',  # Color de las líneas que dividen los rangos del eje Y         
    ),  
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend_title="Trimestre",
    barmode='group',
    legend=dict(
        orientation='h',  # Orientación horizontal de la leyenda
        yanchor='bottom',  # Anclar al borde inferior
        y=1.02,  # Colocar la leyenda justo debajo del gráfico
        xanchor='right',
        x=1
    ),
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)    
    # Configuración personalizada para la gráfica
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

guardar_grafico_como_html(fig, 'g_bar_dist_creditos_segmento_percent_2022', carpeta='assets/graficas')

# Mostrar la gráfica
fig.show()


### 2023

In [16]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica
# Datos originales
data = {
    'segmento': ['E', 'D', 'C', 'B', 'A', 'S', 'L'],
    '1': [7.05, 2.28, 1.01, 0.50, 0.27, 0.64, 0.03],
    '2': [7.59, 1.98, 0.97, 0.64, 0.40, 0.60, 0.13],
    '3': [20.65, 6.31, 2.72, 1.48, 0.84, 1.98, 0.40]
}

# Crear DataFrame
df = pd.DataFrame(data)

# Reorganizar el DataFrame
tabla_porcentajes = df.set_index("segmento")

# Imprimir el DataFrame
print(tabla_porcentajes)


             1     2      3
segmento                   
E         7.05  7.59  20.65
D         2.28  1.98   6.31
C         1.01  0.97   2.72
B         0.50  0.64   1.48
A         0.27  0.40   0.84
S         0.64  0.60   1.98
L         0.03  0.13   0.40


In [17]:


# Definir colores
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']
orden_segmentos=["E", "D", "C", "B","A", "S", "L", 'ELITE']

# Crear la figura de la gráfica
fig = go.Figure()
trimestres = tabla_porcentajes.columns
# Reordenar la tabla de porcentajes según el orden de segmentos
tabla_porcentajes = tabla_porcentajes.reindex(orden_segmentos)

# Agregar trazos para cada trimestre
for i, trimestre in enumerate(trimestres):
    fig.add_trace(go.Bar(
        name=f'{trimestre}', 
        x=tabla_porcentajes.index, 
        y=tabla_porcentajes[trimestre], 
        marker_color=colores[i],
        text=tabla_porcentajes[trimestre],  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    ))

# Actualizar diseño de la gráfica
fig.update_layout(
    yaxis=dict(
        title='%',  # Título del eje y
        gridcolor='#dddcda',  # Color de las líneas que dividen los rangos del eje Y         
    ),  
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend_title="Trimestre",
    barmode='group',
    legend=dict(
        orientation='h',  # Orientación horizontal de la leyenda
        yanchor='bottom',  # Anclar al borde inferior
        y=1.02,  # Colocar la leyenda justo debajo del gráfico
        xanchor='right',
        x=1
    ),
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)    
    # Configuración personalizada para la gráfica
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

guardar_grafico_como_html(fig, 'g_bar_dist_creditos_segmento_percent_2023', carpeta='assets/graficas')

# Mostrar la gráfica
fig.show()
